[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/langchain-ai/langchain-academy/blob/main/module-4/research-assistant.ipynb) [![Open in LangChain Academy](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66e9eba12c7b7688aa3dbb5e_LCA-badge-green.svg)](https://academy.langchain.com/courses/take/intro-to-langgraph/lessons/58239974-lesson-4-research-assistant)

# Research Assistant

## Review

We've covered a few major LangGraph themes:

* Memory
* Human-in-the-loop
* Controllability

Now, we'll bring these ideas together to tackle one of AI's most popular applications: research automation. 

Research is often laborious work offloaded to analysts. AI has considerable potential to assist with this.

However, research demands customization: raw LLM outputs are often poorly suited for real-world decision-making workflows. 

Customized, AI-based [research and report generation](https://jxnl.co/writing/2024/06/05/predictions-for-the-future-of-rag/#reports-over-rag) workflows are a promising way to address this.

## Goal

Our goal is to build a lightweight, multi-agent system around chat models that customizes the research process.

`Source Selection` 
* Users can choose any set of input sources for their research.
  
`Planning` 
* Users provide a topic, and the system generates a team of AI analysts, each focusing on one sub-topic.
* `Human-in-the-loop` will be used to refine these sub-topics before research begins.
  
`LLM Utilization`
* Each analyst will conduct in-depth interviews with an expert AI using the selected sources.
* The interview will be a multi-turn conversation to extract detailed insights as shown in the [STORM](https://arxiv.org/abs/2402.14207) paper.
* These interviews will be captured in a using `sub-graphs` with their internal state. 
   
`Research Process`
* Experts will gather information to answer analyst questions in `parallel`.
* And all interviews will be conducted simultaneously through `map-reduce`.

`Output Format` 
* The gathered insights from each interview will be synthesized into a final report.
* We'll use customizable prompts for the report, allowing for a flexible output format. 

![Screenshot 2024-08-26 at 7.26.33 PM.png](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66dbb164d61c93d48e604091_research-assistant1.png)

## Setup

We'll use [LangSmith](https://docs.langchain.com/langsmith/home) for [tracing](https://docs.langchain.com/langsmith/observability-concepts).

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

## Core Logic and Graph Definitions

In [2]:
import operator
import json
from typing import List, Annotated, TypedDict
from pydantic import BaseModel, Field
from IPython.display import Image, display, Markdown

from langgraph.graph import START, END, StateGraph, MessagesState
from langgraph.checkpoint.memory import MemorySaver
from langgraph.types import Send
from langchain_ollama import ChatOllama
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_tavily import TavilySearch

# --- 1. SETUP ---
# Note: Ensure your TAVILY_API_KEY is set in your environment variables
llm = ChatOllama(model="qwen3:8b", temperature=0)
search_tool = TavilySearch(max_results=3)

class Analyst(BaseModel):
    affiliation: str; name: str; role: str; description: str

class Perspectives(BaseModel):
    analysts: List[Analyst]

class SearchQuery(BaseModel):
    query: str

class InterviewState(MessagesState):
    analyst: Analyst 
    context: Annotated[list, operator.add]
    sections: list
    sources: Annotated[list, operator.add] # New: Track URLs

# --- 2. NODES ---
def ask_question(state: InterviewState):
    analyst = state["analyst"]
    sys_msg = f"You are {analyst.name}, {analyst.role}. Ask a technical question about the topic."
    return {"messages": [llm.invoke([SystemMessage(content=sys_msg)] + state["messages"])]}

def search_node(state: InterviewState):
    """Generates a search query and fetches live data from Tavily."""
    last_msg = state["messages"][-1].content
    sys_msg = SystemMessage(content="Output ONLY a JSON object with a 'query' key for web search.")
    
    # Generate query
    res = llm.with_structured_output(SearchQuery).invoke([sys_msg, HumanMessage(content=last_msg)])
    print(f"🔍 {state['analyst'].name} is researching: {res.query}")
    
    # Execute search
    search_data = search_tool.invoke(res.query)
    
    content_list = []
    source_links = []
    for r in search_data:
        content_list.append(f"Source: {r['url']}\nContent: {r['content']}")
        source_links.append(r['url'])
        
    return {
        "context": ["\n\n".join(content_list)], 
        "sources": source_links
    }

def answer_node(state: InterviewState):
    context_str = "\n".join(state["context"])
    sys_msg = f"Using this research:\n{context_str}\n\nAnswer the analyst's question. Be technical."
    res = llm.invoke([SystemMessage(content=sys_msg)] + state["messages"])
    return {"messages": [res], "sections": [res.content]}

# --- 3. SUB-GRAPH BUILD ---
itv_builder = StateGraph(InterviewState)
itv_builder.add_node("ask", ask_question)
itv_builder.add_node("search", search_node)
itv_builder.add_node("answer", answer_node)
itv_builder.add_edge(START, "ask")
itv_builder.add_edge("ask", "search")
itv_builder.add_edge("search", "answer")
itv_builder.add_edge("answer", END)
interview_graph = itv_builder.compile()

## Execute Workflow

In [3]:
class ResearchState(TypedDict):
    topic: str; max_analysts: int; human_analyst_feedback: str
    analysts: List[Analyst]; sections: Annotated[list, operator.add]
    sources: Annotated[list, operator.add]; final_report: str

def create_analysts(state: ResearchState):
    prompt = f"Create a team of {state['max_analysts']} analysts for: {state['topic']}. Return JSON."
    res = llm.with_structured_output(Perspectives).invoke([SystemMessage(content=prompt)])
    
    # PRINTING IN YOUR REQUESTED FORMAT
    print("\n--- ANALYST TEAM GENERATED ---")
    for a in res.analysts:
        print(f"Name: {a.name}")
        print(f"Affiliation: {a.affiliation}")
        print(f"Role: {a.role}")
        print(f"Description: {a.description}")
        print("-" * 50)
    
    return {"analysts": res.analysts}

def initiate_interviews(state: ResearchState):
    return [Send("conduct_interview", {"analyst": a, "messages": [HumanMessage(content=state["topic"])]}) for a in state["analysts"]]

def compile_report(state: ResearchState):
    all_sections = "\n\n".join(state["sections"])
    # Format sources as a clean Markdown list
    unique_sources = list(set(state.get("sources", [])))
    source_str = "\n".join([f"* [{s}]({s})" for s in unique_sources])
    
    prompt = f"Write a professional report using these sections:\n{all_sections}\n\nAt the very end, add a section called '### References' and list these exact links: {source_str}"
    res = llm.invoke(prompt)
    return {"final_report": res.content}

# GRAPH BUILDER
builder = StateGraph(ResearchState)
builder.add_node("create_analysts", create_analysts)
builder.add_node("human_feedback", lambda s: None)
builder.add_node("conduct_interview", interview_graph)
builder.add_node("write_report", compile_report)

builder.add_edge(START, "create_analysts")
builder.add_edge("create_analysts", "human_feedback")
builder.add_conditional_edges("human_feedback", 
    lambda s: initiate_interviews(s) if s.get("human_analyst_feedback") == "OK" else "create_analysts",
    {"conduct_interview": "conduct_interview", "create_analysts": "create_analysts"})
builder.add_edge("conduct_interview", "write_report")
builder.add_edge("write_report", END)

memory = MemorySaver()
graph = builder.compile(checkpointer=memory, interrupt_before=["human_feedback"])

## Provide Approval and Finalize

In [ ]:
from IPython.display import Markdown, display

config = {"configurable": {"thread_id": "Harris_Research_v2"}, "recursion_limit": 50}

# 1. Start Analysis
initial_input = {"topic": "Best practices for LangGraph parallelization", "max_analysts": 3}
for event in graph.stream(initial_input, config, stream_mode="values"):
    pass

# 2. Approve Analysts (Set to OK to proceed)
graph.update_state(config, {"human_analyst_feedback": "OK"}, as_node="human_feedback")

# 3. Resume with Research (Parallel Processing)
print("\nStarting research and interviews...")
for event in graph.stream(None, config, stream_mode="updates", max_concurrency=1):
    node = list(event.keys())[0]
    print(f"✅ Node {node} completed.")

# 4. Final Display
final_data = graph.get_state(config).values
display(Markdown(final_data.get('final_report', "Report failed to generate.")))

### 4. Professional Formatted Report

In [ ]:
# 1. Extract the report from the final state of the graph
final_state = graph.get_state(config)
report_raw = final_state.values.get('final_report', '')

# 2. Try to parse and format the JSON report
try:
    # Handle the case where the LLM wrapped it in Markdown code blocks
    clean_json = report_raw.replace("```json", "").replace("```", "").strip()
    data = json.loads(clean_json)
    
    # Check if 'report' is the top-level key (as seen in your output)
    content = data.get('report', data)
    
    markdown_out = f"# {content.get('title', 'Research Report')}\n\n"
    markdown_out += f"## Introduction\n{content.get('introduction', '')}\n\n"
    
    for section in content.get('sections', []):
        title = section.get('title') or "Analysis"
        markdown_out += f"### {title}\n{section.get('content', '')}\n\n"
        
    markdown_out += f"## Conclusion\n{content.get('conclusion', '')}\n\n"
    
    if 'references' in content:
        markdown_out += "## References\n* " + "\n* ".join(content['references'])
    
    display(Markdown(markdown_out))

except Exception as e:
    # Fallback: Just print the raw text if JSON parsing fails
    print("Parsing failed or report already in Markdown. Displaying raw output:")
    display(Markdown(report_raw))